In [2]:
from google.colab import drive
drive.mount('/content/drive')
# then set INPUT_PATH to the full drive path, e.g.
# INPUT_PATH = "/content/drive/MyDrive/analyzed_audio_data.xlsx"


Mounted at /content/drive


In [6]:
from google.colab import files
uploaded = files.upload()



Saving analyzed_audio_data.xlsx to analyzed_audio_data.xlsx


In [7]:
{'analyzed_audio_data.xlsx': '...'}


{'analyzed_audio_data.xlsx': '...'}

In [8]:
INPUT_PATH = "analyzed_audio_data.xlsx"

In [12]:
# === Robust Feature engineering cell ===
import pandas as pd
import numpy as np
import re
from functools import reduce

INPUT_PATH = "analyzed_audio_data.xlsx"   # file you just uploaded
OUTPUT_FEATURES = "user_features.csv"

def safe_col(s):
    if pd.isna(s):
        return "unknown"
    s = str(s).strip().lower()
    s = re.sub(r'[^0-9a-z]+', '_', s)
    s = re.sub(r'__+', '_', s).strip('_')
    return s or "unknown"

# Load Excel
xls = pd.ExcelFile(INPUT_PATH)
sheet = xls.sheet_names[0]
df = xls.parse(sheet)
print(f"Loaded {INPUT_PATH} — sheet '{sheet}' — rows: {len(df)} cols: {df.shape[1]}")

# Basic conversions
df['date'] = pd.to_datetime(df['date'], errors='coerce')
df['price'] = pd.to_numeric(df.get('price', 0), errors='coerce').fillna(0)
df['hour'] = df['date'].dt.hour
df['dow'] = df['date'].dt.dayofweek
df['date_only'] = df['date'].dt.date

# Basic per-user aggregates
agg_basic = df.groupby('user').agg(
    total_spend = ('price','sum'),
    num_txns = ('price','count'),
    avg_price = ('price','mean'),
    median_price = ('price','median'),
    min_price = ('price','min'),
    max_price = ('price','max'),
    first_txn = ('date','min'),
    last_txn = ('date','max'),
    active_days = ('date_only', pd.Series.nunique)
).reset_index()

agg_basic['span_days'] = (agg_basic['last_txn'] - agg_basic['first_txn']).dt.days.fillna(0)
agg_basic['txns_per_day'] = agg_basic['num_txns'] / (agg_basic['span_days'].replace(0,1))

# Save engineered features
agg_basic.to_csv(OUTPUT_FEATURES, index=False)
print("✅ Saved user-level features to:", OUTPUT_FEATURES)
agg_basic.head()


Loaded analyzed_audio_data.xlsx — sheet 'Sheet1' — rows: 19205 cols: 17
✅ Saved user-level features to: user_features.csv


,user,total_spend,num_txns,avg_price,median_price,min_price,max_price,first_txn,last_txn,active_days,span_days,txns_per_day
0,user-1,1799,1,1799.000000,1799.0,1799,1799,2024-09-02,2024-09-02,1,0,1.0
1,user-10,3907,3,1302.333333,1409.0,999,1499,2024-09-19,2024-09-24,3,5,0.6
2,user-100,6999,1,6999.000000,6999.0,6999,6999,2024-10-25,2024-10-25,1,0,1.0
3,user-1000,8047,3,2682.333333,599.0,349,7099,2024-09-26,2024-09-28,2,2,1.5
4,user-10000,1099,1,1099.000000,1099.0,1099,1099,2024-09-01,2024-09-01,1,0,1.0


In [13]:
# === Scoring: user_features.csv -> user_features_scored.csv ===
import pandas as pd
import numpy as np

INPUT_FEATURES = "user_features.csv"
OUTPUT_SCORED = "user_features_scored.csv"

# Load user_features (created by previous cell)
uf = pd.read_csv(INPUT_FEATURES, parse_dates=['first_txn','last_txn'], low_memory=False)

# helper to safely access missing cols
def get_col(df, col):
    return df[col] if col in df.columns else pd.Series(0, index=df.index, dtype=float)

# Normalizations to stabilize weights
uf['avg_price_norm'] = (uf['avg_price'] - uf['avg_price'].min()) / (uf['avg_price'].max() - uf['avg_price'].min() + 1e-9)
uf['num_txns_norm'] = (uf['num_txns'] - uf['num_txns'].min()) / (uf['num_txns'].max() - uf['num_txns'].min() + 1e-9)
uf['txns_per_day_norm'] = (uf['txns_per_day'] - uf['txns_per_day'].min()) / (uf['txns_per_day'].max() - uf['txns_per_day'].min() + 1e-9)
uf['total_spend_norm'] = (uf['total_spend'] - uf['total_spend'].min()) / (uf['total_spend'].max() - uf['total_spend'].min() + 1e-9)

col_names = uf.columns.tolist()
def find_pct_col_like(keywords):
    for c in col_names:
        low = c.lower()
        if c.startswith('pct_cat_') and any(k in low for k in keywords):
            return c
    return None

# find category pct columns
pct_beauty_col = next((c for c in col_names if 'pct_beaut' in c), None)
if pct_beauty_col is None and 'pct_beauty' in col_names:
    pct_beauty_col = 'pct_beauty'
apparel_col = find_pct_col_like(['apparel','clothing','fashion'])
footwear_col = find_pct_col_like(['footwear','shoe','shoes'])
audio_video_col = find_pct_col_like(['audio','video','electro','speaker','headset','headphones','camera'])
home_cat_col = find_pct_col_like(['home','kitchen','appliance'])

# brand columns and approximate gender-lean lists (tweak if needed)
brand_cols = [c for c in col_names if c.startswith('pct_brand_')]
male_brands = ['sony','bose','jbl','realme','zebronics','marshall','tribit','boat','oneplus','samsung']
female_brands = ['mivi','mac','maybelline','lakme','loreal','nykaa','fogg']

# compute brand contribution scores
brand_male_score = np.zeros(len(uf))
brand_female_score = np.zeros(len(uf))
for bc in brand_cols:
    lname = bc.replace('pct_brand_','').lower()
    if any(m in lname for m in male_brands):
        brand_male_score += uf[bc].fillna(0).values * 1.2
    if any(f in lname for f in female_brands):
        brand_female_score += uf[bc].fillna(0).values * 1.3

# init scores and components for explainability
male_score = np.zeros(len(uf)); female_score = np.zeros(len(uf))
components = pd.DataFrame(index=uf.index)
components['brand_male_comp'] = brand_male_score
components['brand_female_comp'] = brand_female_score

# beauty -> female
beauty = uf[pct_beauty_col].fillna(0) if pct_beauty_col else get_col(uf, 'pct_beauty').fillna(0)
male_score += 0.1 * beauty
female_score += 2.8 * beauty
components['beauty_comp'] = beauty * 2.8

# apparel & footwear -> female
apparel = get_col(uf, apparel_col).fillna(0) if apparel_col else pd.Series(0, index=uf.index)
footwear = get_col(uf, footwear_col).fillna(0) if footwear_col else pd.Series(0, index=uf.index)
female_score += 1.6 * apparel
female_score += 1.1 * footwear
components['apparel_comp'] = apparel * 1.6
components['footwear_comp'] = footwear * 1.1

# audio/electronics -> male
audio = get_col(uf, audio_video_col).fillna(0) if audio_video_col else get_col(uf,'pct_audio_video').fillna(0)
male_score += 2.0 * audio
components['audio_comp'] = audio * 2.0

# high value -> slight male tilt
male_score += 1.0 * get_col(uf, 'pct_high_value').fillna(0)
components['high_value_comp'] = get_col(uf, 'pct_high_value').fillna(0) * 1.0

# brand comps
male_score += brand_male_score
female_score += brand_female_score
components['brand_male_comp'] = brand_male_score
components['brand_female_comp'] = brand_female_score

# Synergy: beauty x apparel/footwear (use provided synergy if exists)
if 'synergy_beauty_apparel' in uf.columns:
    syn_ba = uf['synergy_beauty_apparel'].fillna(0)
else:
    syn_ba = beauty * apparel
if 'synergy_beauty_footwear' in uf.columns:
    syn_bf = uf['synergy_beauty_footwear'].fillna(0)
else:
    syn_bf = beauty * footwear
female_score += 4.0 * syn_ba + 3.0 * syn_bf
components['syn_beauty_apparel_comp'] = syn_ba * 4.0
components['syn_beauty_footwear_comp'] = syn_bf * 3.0

# night purchases tilt
pct_night = get_col(uf, 'pct_night').fillna(0)
male_score += 0.4 * pct_night
female_score += 0.3 * pct_night
components['pct_night_comp_male'] = pct_night * 0.4
components['pct_night_comp_female'] = pct_night * 0.3

# AGE scoring raw
age_under25 = np.zeros(len(uf)); age_25_40 = np.zeros(len(uf)); age_40_plus = np.zeros(len(uf))
age_under25 += 1.9 * pct_night
age_under25 += 1.2 * uf['num_txns_norm'].fillna(0)
age_under25 += 1.6 * audio
age_under25 += 1.0 * (1 - uf['avg_price_norm'].fillna(0))
components['age_under25_night'] = pct_night * 1.9
components['age_under25_freq'] = uf['num_txns_norm'].fillna(0) * 1.2
components['age_under25_audio'] = audio * 1.6
components['age_under25_price'] = (1 - uf['avg_price_norm'].fillna(0)) * 1.0

age_25_40 += 1.8 * get_col(uf, 'pct_kids').fillna(0)
age_25_40 += 1.0 * apparel
age_25_40 += 0.9 * get_col(uf, 'pct_weekend').fillna(0)
age_25_40 += 0.8 * uf['txns_per_day_norm'].fillna(0)
components['age_25_40_kids'] = get_col(uf, 'pct_kids').fillna(0) * 1.8
components['age_25_40_apparel'] = apparel * 1.0
components['age_25_40_weekend'] = get_col(uf, 'pct_weekend').fillna(0) * 0.9
components['age_25_40_freq'] = uf['txns_per_day_norm'].fillna(0) * 0.8

age_40_plus += 2.2 * get_col(uf, 'pct_high_value').fillna(0)
age_40_plus += 1.6 * uf['avg_price_norm'].fillna(0)
if home_cat_col:
    age_40_plus += 1.3 * get_col(uf, home_cat_col).fillna(0)
age_40_plus += 0.6 * (1 - uf['num_txns_norm'].fillna(0))
components['age_40_high_value'] = get_col(uf, 'pct_high_value').fillna(0) * 2.2
components['age_40_avg_price'] = uf['avg_price_norm'].fillna(0) * 1.6
components['age_40_home'] = get_col(uf, home_cat_col).fillna(0) * 1.3 if home_cat_col else 0
components['age_40_fewer_txns'] = (1 - uf['num_txns_norm'].fillna(0)) * 0.6

# small bias
male_score += 0.01; female_score += 0.01
age_under25 += 0.01; age_25_40 += 0.01; age_40_plus += 0.01
components['male_score_raw'] = male_score; components['female_score_raw'] = female_score

# softmax -> probabilities
def softmax2(a,b):
    maxv = np.maximum(a,b)
    ea = np.exp(a - maxv)
    eb = np.exp(b - maxv)
    s = ea + eb
    return ea/s, eb/s

male_prob, female_prob = softmax2(male_score, female_score)

age_stack = np.vstack([age_under25, age_25_40, age_40_plus]).T
age_max = np.max(age_stack, axis=1)
age_exp = np.exp(age_stack - age_max[:, None])
age_probs = age_exp / (age_exp.sum(axis=1)[:, None] + 1e-12)
age_prob_under25 = age_probs[:,0]; age_prob_25_40 = age_probs[:,1]; age_prob_40_plus = age_probs[:,2]

# assemble and save
uf_scores = uf[['user','total_spend','num_txns','avg_price']].copy()
uf_scores['male_prob'] = male_prob; uf_scores['female_prob'] = female_prob
uf_scores['age_prob_<25'] = age_prob_under25; uf_scores['age_prob_25_40'] = age_prob_25_40; uf_scores['age_prob_40+'] = age_prob_40_plus

# top components for explainability
comp_cols = [c for c in components.columns]
components_df = components.copy(); components_df['user'] = uf['user']
def top_n_components(row, n=3):
    vals = row[comp_cols].values
    idx = np.argsort(vals)[-n:][::-1]
    return ', '.join([f"{comp_cols[i]}({row[comp_cols[i]]:.3f})" for i in idx if row[comp_cols[i]]>0])
components_df['top_3_components'] = components_df.apply(lambda r: top_n_components(r, 3), axis=1)
uf_scores = uf_scores.merge(components_df[['user','top_3_components']], on='user', how='left')

uf_scores.to_csv(OUTPUT_SCORED, index=False)
print("Saved scored users to:", OUTPUT_SCORED)
uf_scores.head(12)


Saved scored users to: user_features_scored.csv


,user,total_spend,num_txns,avg_price,male_prob,female_prob,age_prob_<25,age_prob_25_40,age_prob_40+,top_3_components
0,user-1,1799,1,1799.000000,0.5,0.5,0.466554,0.204478,0.328967,"age_under25_price(0.981), age_40_fewer_txns(0...."
1,user-10,3907,3,1302.333333,0.5,0.5,0.498498,0.189832,0.311670,"age_under25_price(0.986), age_40_fewer_txns(0...."
2,user-100,6999,1,6999.000000,0.5,0.5,0.437864,0.203325,0.358812,"age_under25_price(0.923), age_40_fewer_txns(0...."
3,user-1000,8047,3,2682.333333,0.5,0.5,0.476715,0.213105,0.310180,"age_under25_price(0.971), age_40_fewer_txns(0...."
4,user-10000,1099,1,1099.000000,0.5,0.5,0.470400,0.204565,0.325035,"age_under25_price(0.988), age_40_fewer_txns(0...."
5,user-10001,1998,1,1998.000000,0.5,0.5,0.465460,0.204451,0.330089,"age_under25_price(0.978), age_40_fewer_txns(0...."
6,user-10002,6498,2,3249.000000,0.5,0.5,0.454036,0.229335,0.316629,"age_under25_price(0.964), age_40_fewer_txns(0...."
7,user-10003,1999,1,1999.000000,0.5,0.5,0.465455,0.204450,0.330095,"age_under25_price(0.978), age_40_fewer_txns(0...."
8,user-10004,798,1,798.000000,0.5,0.5,0.472051,0.204598,0.323351,"age_under25_price(0.992), age_40_fewer_txns(0...."
9,user-10005,4047,3,1349.000000,0.5,0.5,0.504699,0.179328,0.315973,"age_under25_price(0.986), age_40_fewer_txns(0...."


In [14]:
from google.colab import files
files.download("user_features_scored.csv")



<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [15]:
uf_scores['male_prob'].value_counts()

,count
male_prob,
0.5,13900


In [16]:
uf_scores.shape

(13900, 10)